## Ucitavanje i Sredjivanje Nizova Podataka

In [118]:
#ucitavanje potrebnih biblioteka
import pandas as pd
import numpy as np

In [119]:
#uvoz podataka
#sezona 23/24
df_23_4 = pd.read_csv("epl-2023.csv")
#sezona 24/25
df_24_5 = pd.read_csv("epl-2024.csv")
#sezona 25/6
df_25_6 = pd.read_csv("epl-2025.csv")

## Ukidanje Duplih Vrijednosti za Nazive Klubova

In [120]:
#za sezonu 25 6
df_25_6["Home Team"] = df_25_6["Home Team"].replace("Nott'm Forest", 'Nottingham Forest')
df_25_6["Away Team"] = df_25_6["Away Team"].replace("Nott'm Forest", 'Nottingham Forest')
#za sezonu 24 5
df_24_5["Home Team"] = df_24_5["Home Team"].replace("Nott'm Forest", 'Nottingham Forest')
df_24_5["Away Team"] = df_24_5["Away Team"].replace("Nott'm Forest", 'Nottingham Forest')

## Određivanje Duzine Trenutnoj Sezoni i Prikaz

In [121]:
#odrediti duzinu sezone - OBAVEZNO!!!
df_25_6 = df_25_6.head(60)
df_25_6.tail(5)

,Match Number,Round Number,Date,Location,Home Team,Away Team,Result
55,59,6,27/09/2025 17:30,The City Ground,Nottingham Forest,Sunderland,0 - 1
56,60,6,27/09/2025 20:00,Tottenham Hotspur Stadium,Spurs,Wolves,1 - 1
57,51,6,28/09/2025 14:00,Villa Park,Aston Villa,Fulham,3 - 1
58,58,6,28/09/2025 16:30,St. James' Park,Newcastle,Arsenal,1 - 2
59,55,6,29/09/2025 20:00,Hill Dickinson Stadium,Everton,West Ham,1 - 1


In [122]:
#sjedinjavanje ta tri df niza podataka - po osi jedan - odnosno uzduž
df_BL = pd.concat([df_23_4, df_24_5, df_25_6], axis=0)

In [123]:
df_BL["Home Team"].unique()

array(['Burnley', 'Arsenal', 'Bournemouth', 'Brighton', 'Everton',
       'Sheffield Utd', 'Newcastle', 'Brentford', 'Chelsea', 'Man Utd',
       'Nottingham Forest', 'Fulham', 'Liverpool', 'Wolves', 'Spurs',
       'Man City', 'Aston Villa', 'West Ham', 'Crystal Palace', 'Luton',
       'Ipswich', 'Leicester', 'Southampton', 'Sunderland', 'Leeds'],
      dtype=object)

In [124]:
#prikaz sjedinjenog niza podatak 
#tj. sve tri sezone u jednom nizu podataka u obliku df 
df_BL.tail()

,Match Number,Round Number,Date,Location,Home Team,Away Team,Result
55,59,6,27/09/2025 17:30,The City Ground,Nottingham Forest,Sunderland,0 - 1
56,60,6,27/09/2025 20:00,Tottenham Hotspur Stadium,Spurs,Wolves,1 - 1
57,51,6,28/09/2025 14:00,Villa Park,Aston Villa,Fulham,3 - 1
58,58,6,28/09/2025 16:30,St. James' Park,Newcastle,Arsenal,1 - 2
59,55,6,29/09/2025 20:00,Hill Dickinson Stadium,Everton,West Ham,1 - 1


## Predprocesovanje Niza Podataka

In [125]:
#Izbacivanje Nerelevantnih Osobina
df_BL = df_BL.drop(["Match Number","Round Number","Date","Location"],axis=1)
df_BL.shape

(820, 3)

In [126]:
df_BL.rename(columns={"Home Team":"H","Away Team":"A"},inplace=True)

In [127]:
df_BL.to_csv("epl.csv")
df_BL

,H,A,Result
0,Burnley,Man City,0 - 3
1,Arsenal,Nottingham Forest,2 - 1
2,Bournemouth,West Ham,1 - 1
3,Brighton,Luton,4 - 1
4,Everton,Fulham,0 - 1
...,...,...,...
55,Nottingham Forest,Sunderland,0 - 1
56,Spurs,Wolves,1 - 1
57,Aston Villa,Fulham,3 - 1
58,Newcastle,Arsenal,1 - 2


In [128]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder
import pickle

#One Hot Encoder Timova - Domacin i Gost
onehot_encoder_domacin = OneHotEncoder(handle_unknown="ignore",drop="first")  # Ensure parameters are correct
kodiran_domacin = onehot_encoder_domacin.fit_transform(df_BL[["H"]]).toarray()
df_domacin = pd.DataFrame(kodiran_domacin,columns=onehot_encoder_domacin.get_feature_names_out(["H"]))

In [129]:
df_domacin.columns

Index(['H_Aston Villa', 'H_Bournemouth', 'H_Brentford', 'H_Brighton',
       'H_Burnley', 'H_Chelsea', 'H_Crystal Palace', 'H_Everton', 'H_Fulham',
       'H_Ipswich', 'H_Leeds', 'H_Leicester', 'H_Liverpool', 'H_Luton',
       'H_Man City', 'H_Man Utd', 'H_Newcastle', 'H_Nottingham Forest',
       'H_Sheffield Utd', 'H_Southampton', 'H_Spurs', 'H_Sunderland',
       'H_West Ham', 'H_Wolves'],
      dtype='object')

In [130]:
#One Hot Encoder Timova - Gosti
onehot_encoder_gost = OneHotEncoder(handle_unknown="ignore",drop="first")  # Ensure parameters are correct
kodiran_gost = onehot_encoder_gost.fit_transform(df_BL[["A"]]).toarray()
df_gost = pd.DataFrame(kodiran_gost,columns=onehot_encoder_gost.get_feature_names_out(["A"]))

In [131]:
#Sjedinjavanje tih osobina domacin i gost
df_jedan = pd.concat([df_domacin,df_gost],axis=1)
df_jedan

,H_Aston Villa,H_Bournemouth,H_Brentford,H_Brighton,H_Burnley,H_Chelsea,H_Crystal Palace,H_Everton,H_Fulham,H_Ipswich,...,A_Man City,A_Man Utd,A_Newcastle,A_Nottingham Forest,A_Sheffield Utd,A_Southampton,A_Spurs,A_Sunderland,A_West Ham,A_Wolves
0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
815,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
816,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
817,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
818,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## Transformisanje Rezultata u Varijablu - Y Varijable

#### Stvaranje Vrijednosti za Dvije Kolone iz Jedne Osobine/Kolone Rezultat

In [132]:
rez_dom = df_BL["Result"].str.slice(0,1)

In [133]:
rez_dom.head()

0    0
1    2
2    1
3    4
4    0
Name: Result, dtype: object

In [134]:
rez_gost = df_BL["Result"].str.slice(4,5)

In [135]:
rez_gost.head()

0    3
1    1
2    1
3    1
4    1
Name: Result, dtype: object

In [136]:
df_rezultatski = pd.concat([df_BL,rez_dom,rez_gost],axis=1)

In [137]:
#promjena naziva kolonama - spajanjem smo dobili kolone istih naziva
df_rezultatski.columns = ["H","A","Result","Rez1","Rez2"]
df_rezultatski.tail()

,H,A,Result,Rez1,Rez2
55,Nottingham Forest,Sunderland,0 - 1,0,1
56,Spurs,Wolves,1 - 1,1,1
57,Aston Villa,Fulham,3 - 1,3,1
58,Newcastle,Arsenal,1 - 2,1,2
59,Everton,West Ham,1 - 1,1,1


In [138]:
#spremanje df_epl varijable za obradu osobine Result u excelu
df_rezultatski.to_csv("uredjivanje_skora_vani.csv")

## Uocavanje Indeksa Sezona - za Kasnije Floatiranje

In [139]:
#df_rezultatski[0:380] #sezona 23/4 od 0:380
#df_rezultatski[380:759] #sezona 24/5 od 380:759
#df_rezultatski[760:] #sezona25/6 od 760:

## Stvaranje Kolone Skor sa Vrijednostima - Ovdje

In [140]:
# Define a function to perform the conditional logic
def calculate_value(row):
    if row['Rez1'] > row['Rez2']:
        return 1
    elif row['Rez1'] < row['Rez2']:
        return -1
    else:  # row['Column1'] == row['Column2']
        return 0

In [141]:
# Apply the function to each row in the DataFrame and assign the result to a new column
df_rezultatski['Skor'] = df_rezultatski.apply(calculate_value, axis=1)
skor = df_rezultatski.drop(['H', 'A', 'Result', 'Rez1',"Rez2"],axis=1)

### Floatiranje Vrijednosti Kolone Skor - po Sezonama

In [142]:
#sezona 23/4
skor23_4 = skor[0:380]
skor24_5 = skor[380:760]
skor25_6 = skor[760:]

In [143]:
#skor.head(3)
skor23_4["Skor"] = skor23_4["Skor"] * 0.71

C:\Users\amirb\AppData\Local\Temp\ipykernel_5780\3544292283.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  skor23_4["Skor"] = skor23_4["Skor"] * 0.71


In [144]:
skor24_5["Skor"] = skor24_5["Skor"] * 0.85

C:\Users\amirb\AppData\Local\Temp\ipykernel_5780\3136881136.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  skor24_5["Skor"] = skor24_5["Skor"] * 0.85


## Konačna kolona Skor - Sjedinjavanje Vrijednosti Sezona

In [145]:
skor = pd.concat([skor23_4, skor24_5, skor25_6], axis=0, ignore_index=True)

In [146]:
df_rezultatski = df_rezultatski.reset_index(drop=True)
skor = skor.reset_index(drop=True)

In [147]:
#provjera rezultata
finalna_provjera = pd.concat([df_rezultatski,skor],axis=1,ignore_index=True)
finalna_provjera = finalna_provjera.drop([3,4,5],axis=1)
finalna_provjera.columns = ["H","A","Rezultat","Skor"]

In [148]:
finalna_provjera[-5:-1]

,H,A,Rezultat,Skor
815,Nottingham Forest,Sunderland,0 - 1,-1.0
816,Spurs,Wolves,1 - 1,0.0
817,Aston Villa,Fulham,3 - 1,1.0
818,Newcastle,Arsenal,1 - 2,-1.0


# Konačni df - Podaci su Sređeni

In [149]:
skor = skor.reset_index(drop=True)
df_jedan = df_jedan.reset_index(drop=True)

In [150]:
df_BL = pd.concat([df_jedan,skor],axis=1)

In [151]:
df_BL.head(5)

,H_Aston Villa,H_Bournemouth,H_Brentford,H_Brighton,H_Burnley,H_Chelsea,H_Crystal Palace,H_Everton,H_Fulham,H_Ipswich,...,A_Man Utd,A_Newcastle,A_Nottingham Forest,A_Sheffield Utd,A_Southampton,A_Spurs,A_Sunderland,A_West Ham,A_Wolves,Skor
0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.71
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.71
2,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.00
3,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.71
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.71


In [152]:
#spremanje kodiranog niza podataka
df_BL.to_csv("kodirani_epl.csv")

## Razdvajanje Niza Podataka na Osobine za Predvidjanje i Ciljanu Osobinu

In [153]:
# X predvidjacke, y varijabla jeste Skor Osobina
X = df_BL.drop(["Skor"],axis=1)
y = df_BL["Skor"]

In [154]:
#razdvajanje podataka u treniranju i testirajuce nizove
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.1,random_state=42)
X_train.head()

,H_Aston Villa,H_Bournemouth,H_Brentford,H_Brighton,H_Burnley,H_Chelsea,H_Crystal Palace,H_Everton,H_Fulham,H_Ipswich,...,A_Man City,A_Man Utd,A_Newcastle,A_Nottingham Forest,A_Sheffield Utd,A_Southampton,A_Spurs,A_Sunderland,A_West Ham,A_Wolves
299,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
440,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
404,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
72,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
77,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [155]:
#skaliranje ovih osobina
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)
X_train

array([[-0.22974277, -0.23299295, -0.22645541, ..., -0.03683547,
        -0.23620752, -0.21976227],
       [-0.22974277, -0.23299295, -0.22645541, ..., -0.03683547,
        -0.23620752, -0.21976227],
       [-0.22974277, -0.23299295, -0.22645541, ..., -0.03683547,
        -0.23620752, -0.21976227],
       ...,
       [-0.22974277, -0.23299295, -0.22645541, ..., -0.03683547,
        -0.23620752, -0.21976227],
       [-0.22974277, -0.23299295, -0.22645541, ..., -0.03683547,
        -0.23620752, -0.21976227],
       [-0.22974277, -0.23299295, -0.22645541, ..., -0.03683547,
        -0.23620752, -0.21976227]])

In [156]:
#Spremanje tog Kodera - za Kasniju Upotrebu

with open("onehot_encoder_domacin.pkl","wb") as file:
    pickle.dump(onehot_encoder_domacin,file)
    
with open("onehot_encoder_gost.pkl","wb") as file:
    pickle.dump(onehot_encoder_gost,file)
    
with open("scaler.pkl","wb") as file:
    pickle.dump(scaler,file)
    

## Stvaranje Modela i Dobijanje Predikcije

In [157]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
#sledeca biblioteka se odnosi na stvaranje neurona iz kutije layers i alatom dense
from tensorflow.keras.layers import Dense

In [158]:
#konstrukcija modela
model=Sequential([
    Dense(16,activation="relu",input_shape=(X_train.shape[1],)), #HL1 connected to I/p - Skriveni Sloj 1 povezan sa I/p
    Dense(1,activation="tanh")
])

In [159]:
#Oblikovanje Modela 
model.compile(optimizer="adam",loss="mean_absolute_error",metrics=["mae"])
model.summary()

# Keras dokumentacija pojasnjava sta se dogodi sa razlicitim odabirom "loss" i drugih parametara

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_4 (Dense)             (None, 16)                784       
                                                                 
 dense_5 (Dense)             (None, 1)                 17        
                                                                 
Total params: 801
Trainable params: 801
Non-trainable params: 0
_________________________________________________________________


In [160]:
import datetime
#uspostavljanje Tensorboard-a, zapocinjanje treniranja
#TensorBoard sluzi samo za vizualizaciju svih logovanja koje izvrsim kod treniranja nekog modela
from tensorflow.keras.callbacks import EarlyStopping,TensorBoard

#log_dir varijabla koja osigurava da ostane zabinjezen trag svakog treniranja modela
log_dir = "regressionlogs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorflow_callback = TensorBoard(log_dir=log_dir,histogram_freq=1)

#Uspostavljanje Early Stopping - sluzi za zaustavljanje treniranja ako se vrijednost gubitaka ne smanjuje
#bitni parametri su uneseni koji sluze poboljsanju funkcije ranog zaustavljanja treniranja modela...
early_stopping_callback = EarlyStopping(monitor="val_loss",patience=10,restore_best_weights=True)


In [161]:
#TRENIRANJE MODELA / Tovarenje Modela
from tensorflow.keras.callbacks import EarlyStopping,TensorBoard

history = model.fit(
    X_train,y_train,validation_data=(X_test,y_test),epochs=50,
    callbacks= [tensorflow_callback,early_stopping_callback]
)

Epoch 1/50
24/24 [==============================] - 33s 312ms/step - loss: 0.8755 - mae: 0.8755 - val_loss: 0.8450 - val_mae: 0.8450
Epoch 2/50
24/24 [==============================] - 2s 107ms/step - loss: 0.8114 - mae: 0.8114 - val_loss: 0.7914 - val_mae: 0.7914
Epoch 3/50
24/24 [==============================] - 2s 66ms/step - loss: 0.7505 - mae: 0.7505 - val_loss: 0.7482 - val_mae: 0.7482
Epoch 4/50
24/24 [==============================] - 6s 239ms/step - loss: 0.6980 - mae: 0.6980 - val_loss: 0.7152 - val_mae: 0.7152
Epoch 5/50
24/24 [==============================] - 3s 120ms/step - loss: 0.6499 - mae: 0.6499 - val_loss: 0.6890 - val_mae: 0.6890
Epoch 6/50
24/24 [==============================] - 3s 127ms/step - loss: 0.6125 - mae: 0.6125 - val_loss: 0.6801 - val_mae: 0.6801
Epoch 7/50
24/24 [==============================] - 1s 53ms/step - loss: 0.5803 - mae: 0.5803 - val_loss: 0.6723 - val_mae: 0.6723
Epoch 8/50
24/24 [==============================] - 1s 46ms/step - loss: 0.55

In [162]:
#Procjena Rada Modela preko test podataka
test_loss, test_mae = model.evaluate(X_test,y_test)

3/3 [==============================] - 0s 5ms/step - loss: 0.6147 - mae: 0.6147


In [163]:
#spremanje modela
model.save("epl_float_model.h5")

In [164]:
#moze biti da skaliranje je neophodno, onda je potrebno i obnoviti test parametara modela, te iznova
#-trenirati model i na kraju ga ponovo spremiti.